In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/cancokisler/Downloads/Datasets/Moral Machine Data/SharedResponses.csv", nrows=100000)

In [3]:
df_dropped = df.dropna()

In [4]:
df_dropped.to_csv("moral_machine_small_sample.csv")

In [5]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_401"; Java(TM) SE Runtime Environment (build 1.8.0_401-b10); Java HotSpot(TM) 64-Bit Server VM (build 25.401-b10, mixed mode)
  Starting server from /Users/cancokisler/anaconda3/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/l4/9k9twr_s74bfv6841pgf6nmc0000gn/T/tmp4lstes76
  JVM stdout: /var/folders/l4/9k9twr_s74bfv6841pgf6nmc0000gn/T/tmp4lstes76/h2o_cancokisler_started_from_python.out
  JVM stderr: /var/folders/l4/9k9twr_s74bfv6841pgf6nmc0000gn/T/tmp4lstes76/h2o_cancokisler_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Toronto
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 27 days
H2O_cluster_name:,H2O_from_python_cancokisler_rdlz8j
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [6]:
df_new = df_dropped.drop(['ResponseID', 'ExtendedSessionID', 'UserID', 'Template'], axis=1)

In [7]:
df_new['AttributeLevel'] = df_new['AttributeLevel'].astype('category')
df_new['ScenarioTypeStrict'] = df_new['ScenarioTypeStrict'].astype('category')
df_new['ScenarioType'] = df_new['ScenarioType'].astype('category')
df_new['DefaultChoice'] = df_new['DefaultChoice'].astype('category')
df_new['UserCountry3'] = df_new['UserCountry3'].astype('category')
df_new['NonDefaultChoice'] = df_new['NonDefaultChoice'].astype('category')

In [8]:
df_new.to_csv("moral_machine_small_sample_h2o.csv")

In [9]:
h2o_df = h2o.import_file("moral_machine_small_sample_h2o.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
import numpy as np
from h2o.estimators import H2ODecisionTreeEstimator
target_variable = 'Saved'
h2o_df[target_variable] = h2o_df[target_variable].asfactor()

# Split the dataset into train and test:
train, test = h2o_df.split_frame(ratios=[.7])

# Build and train the model:
sdt_h2o = H2ODecisionTreeEstimator(model_id="decision_tree.hex", max_depth=4)
sdt_h2o.train(y=target_variable, training_frame=train)

# Predict on the test data:
pred_test = sdt_h2o.predict(test)

dt Model Build progress: |

/Users/cancokisler/anaconda3/lib/python3.11/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [Intervention]
  warnings.warn(mesg["message"], RuntimeWarning)


███████████████████████████████████████████████████████| (done) 100%
dt prediction progress: |████████████████████████████████████████████████████████| (done) 100%


In [11]:
test.describe()

Rows:23244
Cols:38

,C1,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,NonDefaultChoice,DefaultChoiceIsOmission,NumberOfCharacters,DiffNumberOFCharacters,Saved,DescriptionShown,LeftHand,UserCountry3,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
type,int,int,int,int,int,int,enum,enum,enum,enum,enum,int,int,int,enum,int,int,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int
mins,7.0,1.0,0.0,0.0,0.0,0.0,,,,,,0.0,1.0,0.0,,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,49873.38061435209,6.725047324040613,0.0,0.48051109963861643,0.19897607984856308,0.6743675787299949,,,,,,0.4898898640509379,3.1245482705214247,0.4983651695061091,,0.2731027361899845,0.5470228876269144,,0.352392015143693,0.34817587334365857,0.05235759765961108,0.05425055928411633,0.20073997590776113,0.19970745138530374,0.15930992944415767,0.162192393736018,0.09387368783341937,0.16395628979521604,0.16236448115642751,0.054723799690242644,0.10135949062123559,0.10497332644983652,0.1858544140423335,0.18172431595250388,0.09223885733952848,0.09344346928239546,0.1805627258647393,0.18034761658922732
maxs,99983.0,13.0,0.0,1.0,1.0,2.0,,,,,,1.0,5.0,4.0,,1.0,1.0,,5.0,5.0,3.0,3.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,3.0,4.0,3.0,5.0,5.0,4.0,4.0,5.0,5.0
sigma,28928.215229277543,3.776529741671289,0.0,0.4996307860291181,0.3992385963466219,0.8572381810899052,,,,,,0.4999085283128758,1.4620562337986545,1.1166706766143628,,0.44556275948485813,0.49779464524419115,,0.6288862142683987,0.6282241677941367,0.24093787724497748,0.24582650227890213,0.5435611561492506,0.5400517538698988,0.4540152164559889,0.45304092711634697,0.37446157265462937,0.4548241127323832,0.4519332976786853,0.24859380168067235,0.3442946232460062,0.3488054416633906,0.5227315217488289,0.5141543625111942,0.3257403568795081,0.32894737490330817,0.5924120515940609,0.5829240860473464
zeros,0,0,23244,12075,18619,13557,,,,,,11857,0,18589,,16896,10529,,16713,16796,22120,22083,19803,19801,20254,20181,21539,20135,20162,22084,21175,21105,20029,20077,21341,21322,20694,20612
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,7.0,6.0,0.0,0.0,0.0,2.0,Hoomans,Species,Species,Hoomans,Pets,1.0,5.0,0.0,1,1.0,0.0,CAN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,11.0,2.0,0.0,0.0,0.0,2.0,Fit,Fitness,Fitness,Fit,Fat,1.0,4.0,0.0,1,0.0,1.0,USA,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13.0,4.0,0.0,0.0,0.0,2.0,Old,Age,Age,Young,Old,0.0,2.0,0.0,0,1.0,1.0,GBR,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
pred_test[0] == test['Saved']

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Name lookup of 'py_19_sid_8420' failed
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_23_sid_8420 (== (cols_py py_19_sid_8420 0) (cols_py py_4_sid_8420 'Saved')))", 'session_id': '_sid_8420'}


H2OResponseError: Server error water.exceptions.H2OKeyNotFoundArgumentException:
  Error: Object 'py_23_sid_8420' not found for argument: key
  Request: GET /3/Frames/py_23_sid_8420
    params: {'row_count': '10', 'row_offset': '0', 'column_count': '-1', 'full_column_count': '-1', 'column_offset': '0'}


In [34]:
test

C1,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,NonDefaultChoice,DefaultChoiceIsOmission,NumberOfCharacters,DiffNumberOFCharacters,Saved,DescriptionShown,LeftHand,UserCountry3,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
7,6,0,0,0,2,Hoomans,Species,Species,Hoomans,Pets,1,5,0,1,1,0,CAN,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0
11,2,0,0,0,2,Fit,Fitness,Fitness,Fit,Fat,1,4,0,1,0,1,USA,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,4,0,0,0,2,Old,Age,Age,Young,Old,0,2,0,0,1,1,GBR,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,3,0,1,0,2,Less,Utilitarian,Utilitarian,More,Less,0,2,3,0,0,1,TUR,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
31,6,0,1,0,0,Old,Age,Age,Young,Old,0,3,0,0,0,1,CAN,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32,13,0,0,0,2,More,Utilitarian,Utilitarian,More,Less,1,5,1,0,1,1,USA,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0
34,12,0,0,1,0,Pets,Species,Species,Hoomans,Pets,0,5,0,0,1,1,BRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1
40,4,0,0,0,0,Hoomans,Species,Species,Hoomans,Pets,1,1,0,1,1,0,USA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
44,2,0,0,0,0,Less,Utilitarian,Utilitarian,More,Less,0,2,3,1,0,0,USA,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,1,0,0,1,0,Fit,Fitness,Fitness,Fit,Fat,1,5,0,1,1,1,USA,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0
